In [2]:
from numpy.random import RandomState
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import jieba
import jieba.analyse

df = pd.read_csv('movies.csv')
#str -> dict
df['類別'] = df['類別'].apply(eval)

# list first catagory
def list_genres(x):
    for d in x.values():
        return(d)

df['類別'] = df['類別'].apply(list_genres)

In [3]:
#split overview
overview = []
for d in df['劇情介紹']:
    keywords = jieba.analyse.extract_tags(d, topK=5)
    overview.append(keywords)

df['關鍵字'] = overview
df['關鍵字'].head()


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/g9/__yhhv7d0xx59_jssnmpjhwh0000gn/T/jieba.cache
Loading model cost 0.773 seconds.
Prefix dict has been built successfully.


0            [電影, 這部, 小小, 動畫, 超級]
1           [蝙蝠, 一個, 布魯斯, 韋恩, 並從]
2          [奧斯卡, 雷托領, 治療, 實驗, 結果]
3    [捍衛戰士, 任務, 1986, 湯姆克魯斯, 獨行俠]
4           [魔鬼, 星未來, 童星, 保羅, 續集]
Name: 關鍵字, dtype: object

In [4]:
import re
df['keywords'] = ''
def bag_words(x):
    return (''.join(re.sub('[^\u4e00-\u9fa5]+', '', x['中文名稱'])) + ' ' + ''.join(x['類別']) + ' ' +  ' '.join(x['關鍵字']))
df['keywords'] = df.apply(bag_words, axis = 1)

df['keywords'].head()

0           小小兵格魯的崛起 動畫 電影 這部 小小 動畫 超級
1               蝙蝠俠 動作 蝙蝠 一個 布魯斯 韋恩 並從
2              魔比斯 動作 奧斯卡 雷托領 治療 實驗 結果
3    捍衛戰士獨行俠 動作 捍衛戰士 任務 1986 湯姆克魯斯 獨行俠
4           魔鬼剋星未來世 動作 魔鬼 星未來 童星 保羅 續集
Name: keywords, dtype: object

In [5]:
#split Data to train and test
rng = RandomState()
train = df.sample(frac=0.9173, random_state=rng)
test = df.loc[~df.index.isin(train.index)]
test['類別'].to_csv('test.csv')

In [6]:
# 將文件中的詞語轉換為詞頻矩陣
cv = CountVectorizer()
cv_train = cv.fit_transform(train['keywords'])
cv_test = cv.transform(test['keywords'])
#print(cv_train[0:3])

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer
#類調用
transformer = TfidfTransformer()
#將詞頻矩陣X統計成TF-IDF值
tfidf_train = transformer.fit_transform(cv_train)
tfidf_test = transformer.transform(cv_test)
#print(tfidf)

In [8]:
from sklearn.neighbors import KNeighborsClassifier
knn  = KNeighborsClassifier(n_neighbors=9,algorithm='brute',n_jobs=-1)
knn.fit(tfidf_train,train['類別'])

KNeighborsClassifier(algorithm='brute', n_jobs=-1, n_neighbors=9)

In [9]:
knn.score(tfidf_train,train['類別'])

0.7963564213564214

In [10]:
knn.predict(tfidf_test)

array(['動作', '劇情', '動作', '動作', '動畫', '劇情', '戰爭', '動作', '動作', '動作', '劇情',
       '動作', '喜劇', '動作', '動作', '劇情', '動作', '動作', '冒險', '劇情', '愛情', '劇情',
       '戰爭', '冒險', '動作', '動作', '動作', '動作', '動作', '動作', '劇情', '動作', '劇情',
       '動作', '動作', '動作', '冒險', '動作', '動作', '劇情', '動作', '動畫', '動作', '動作',
       '動作', '動作', '動作', '動作', '動作', '劇情', '動作', '劇情', '動作', '動作', '劇情',
       '動作', '動畫', '動作', '戰爭', '動作', '動畫', '動作', '動作', '動作', '溫馨/家庭',
       '劇情', '動作', '冒險', '冒險', '動畫', '冒險', '劇情', '溫馨/家庭', '歷史/傳記', '冒險',
       '動畫', '科幻', '冒險', '冒險', '冒險', '劇情', '劇情', '劇情', '動作', '動作', '劇情',
       '冒險', '冒險', '動作', '冒險', '劇情', '動作', '冒險', '動作', '動作', '動作', '動作',
       '劇情', '冒險', '冒險', '動作', '動作', '動作', '冒險', '動作', '科幻', '科幻', '動畫',
       '恐怖', '劇情', '動作', '科幻', '科幻', '劇情', '科幻', '劇情', '動作', '恐怖', '動作',
       '動作', '冒險', '恐怖', '愛情', '劇情', '劇情', '奇幻', '奇幻', '劇情', '奇幻', '奇幻',
       '奇幻', '動作', '劇情', '動作', '喜劇', '奇幻', '劇情', '恐怖', '劇情', '歷史/傳記',
       '劇情', '恐怖', '劇情', '喜劇', '劇情', '劇情', '劇情', '劇情', '劇

In [11]:
Accuracy = knn.score(tfidf_test,test['類別'])
print(Accuracy*100 , '%')

70.8 %
